CNN_CV_Coursework.ipynb/Code

In [17]:
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage import img_as_ubyte, io, color
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm, metrics
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from PIL import Image
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from google.colab.patches import cv2_imshow
import torch.optim as optim
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import pickle as pkl
from google.colab.patches import cv2_imshow

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)


%matplotlib inline

cuda:1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CV_Coursework/CW_Folder_PG/CW_Folder_PG' 
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['.DS_Store', 'CW_Dataset', 'Video', 'Models', 'Code', 'test_functions.ipynb']


In [4]:
zip_path = "/content/drive/MyDrive/CV_Coursework/CW_Folder_PG/CW_Folder_PG/CW_Dataset/CW_Dataset.zip"
!cp "{zip_path}" .
!yes|unzip -q CW_Dataset.zip

In [5]:
# Reference : https://localcoder.org/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader - solution 2

class MyDataset(Dataset):
    def __init__(self, dataframe, targets, transform=None):
        self.dataframe = dataframe
        self.targets = targets
        self.transform = transform
    def __len__(self):
        return len(self.dataframe)    
    def __getitem__(self, index):
        if torch.is_tensor(index):
          index = index.tolist()
        
        ## Add the path of the images
        im_path = os.path.join(self.targets, self.dataframe.iloc[index, 0])
        im = Image.open(im_path)
        im_label = self.dataframe.iloc[index, 1]
        if self.transform:
            im = self.transform(im)
        
        return im, im_label
     

In [6]:
train_txt = "/content/labels/list_label_train.txt"
test_txt = "/content/labels/list_label_test.txt"

In [7]:
dataframe_CNN = ["Image","Label"]

In [8]:
label_train = pd.read_csv(train_txt, names = dataframe_CNN,delim_whitespace = True)
label_test = pd.read_csv(test_txt, names = dataframe_CNN,delim_whitespace = True)

In [9]:
label_train["Image"] = label_train["Image"].apply(lambda i: i[:-4])
label_test["Image"] = label_test["Image"].apply(lambda i: i[:-4])

label_train["Image"] = label_train["Image"] + "_aligned.jpg"
label_test["Image"] = label_test["Image"] + "_aligned.jpg"

In [ ]:
label_train

,Image,Label
0,train_00001_aligned.jpg,5
1,train_00002_aligned.jpg,5
2,train_00003_aligned.jpg,4
3,train_00004_aligned.jpg,4
4,train_00005_aligned.jpg,5
...,...,...
12266,train_12267_aligned.jpg,7
12267,train_12268_aligned.jpg,7
12268,train_12269_aligned.jpg,7
12269,train_12270_aligned.jpg,7


In [ ]:
label_train['Label'].value_counts()

4    4772
7    2524
5    1982
1    1290
3     717
6     705
2     281
Name: Label, dtype: int64

In [12]:
## Reference: Lab 8

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [13]:
# REFERENCE: Lab 8
# Creating our train and test images and applying them to our DataLoader
train_image_path = "/content/train"
test_image_path = "/content/test"

train_img = MyDataset(dataframe = label_train, targets = train_image_path, transform = transform)

test_img = MyDataset(dataframe = label_test,targets = test_image_path, transform = transform)

trainloader = torch.utils.data.DataLoader(train_img, batch_size=64,
                                          shuffle=True, num_workers=2)


testloader = torch.utils.data.DataLoader(test_img, batch_size=64,
                                         shuffle=False, num_workers=2)

In [14]:
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
## and REFERENCE: Lab 8 
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 22 * 22 , 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 8)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), 16 * 22 *22)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
import time

t0 = time.time()
cnn_loss_list = []
cnn_accuracy_list = []
for epoch in range(20):  # loop over the training set two times

    running_loss = 0.0
    for i, (images, labels) in enumerate(trainloader, 0):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics (loss.item() returns the mean loss in the mini-batch)
        running_loss += loss.item()
        if i % 100:
          print(loss)
          running_loss = 0.0
print('Finished Training: total time in seconds =', time.time() - t0)

tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.0104, grad_fn=<NllLossBackward0>)
tensor(0.0043, grad_fn=<NllLossBackward0>)
tensor(0.0090, grad_fn=<NllLossBackward0>)
tensor(0.0034, grad_fn=<NllLossBackward0>)
tensor(0.0065, grad_fn=<NllLossBackward0>)
tensor(0.0066, grad_fn=<NllLossBackward0>)
tensor(0.0058, grad_fn=<NllLossBackward0>)
tensor(0.0053, grad_fn=<NllLossBackward0>)
tensor(0.0039, grad_fn=<NllLossBackward0>)
tensor(0.0055, grad_fn=<NllLossBackward0>)
tensor(0.0061, grad_fn=<NllLossBackward0>)
tensor(0.0069, grad_fn=<NllLossBackward0>)
tensor(0.0040, grad_fn=<NllLossBackward0>)
tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.0059, grad_fn=<NllLossBackward0>)
tensor(0.0076, grad_fn=<NllLossBackward0>)
tensor(0.0034, grad_fn=<NllLossBackward0>)
tensor(0.0097, grad_fn=<NllLossBackward0>)
tensor(0.0068, grad_fn=<NllLossBackward0>)
tensor(0.0056, grad_fn=<NllLossBackward0>)
tensor(0.0072, grad_fn=<NllLossBackward0>)
tensor(0.0051, grad_fn=<NllLossBackward0>)
tensor(0.00

In [ ]:
Model_path = "/content/drive/MyDrive/CV_Coursework/CW_Folder_PG/CW_Folder_PG/Models/CNN_FR.pt"

torch.save(net.state_dict(),Model_path)

In [10]:
# AlexNet 

# Initating our AlexNet as well as our optimisation and loss function

alexnet_cnn = models.alexnet(pretrained = True)
criterion = nn.CrossEntropyLoss().cuda()
optimizer_ft = optim.SGD(alexnet_cnn.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [ ]:
# AlexNet
## REFERENCE: https://discuss.pytorch.org/t/how-does-enumerate-trainloader-0-work/14410 (using the for loop for training and testing)
## REFERENCE: Combined the testing and training from Lab 8
# In this loop, we will train and test our AlexNet and evaluate it's accuracy

m = 0
alexnet_accuracy_list = []
epochs = 30

for epoch in range(epochs):
    for i, (images, labels) in enumerate(trainloader):
            alexnet_cnn = alexnet_cnn.cuda()

            optimizer_ft.zero_grad()
            outputs = alexnet_cnn(images.cuda())
            alexnet_loss = criterion(outputs.cuda(), labels.cuda())
            alexnet_loss.backward()
            optimizer_ft.step()

            m += 1

            if m % 50 == 0:
                correct = 0
                total = 0

                for i, (images, labels) in enumerate(testloader):

                    outputs = alexnet_cnn(images.cuda())
                    predicted = torch.max(outputs.data, 1)[1]
                    total += len(labels)
                    correct += (predicted == labels.cuda()).sum()
                alexnet_accuracy = 100 * correct / float(total)

                alexnet_accuracy_list.append(alexnet_accuracy)

                if m % 500 == 0:
                    print("AlexNet Accuracy: {} %".format(alexnet_accuracy))

AlexNet Accuracy: 62.15775680541992 %
AlexNet Accuracy: 70.37157440185547 %
AlexNet Accuracy: 69.85006713867188 %
AlexNet Accuracy: 73.2072982788086 %
AlexNet Accuracy: 71.02346801757812 %
AlexNet Accuracy: 73.85919189453125 %
AlexNet Accuracy: 72.06649017333984 %
AlexNet Accuracy: 74.83702850341797 %
AlexNet Accuracy: 74.08734893798828 %
AlexNet Accuracy: 73.85919189453125 %
AlexNet Accuracy: 73.43546295166016 %


In [ ]:
import pickle 
alex_net = "/content/drive/MyDrive/CV_Coursework/CW_Folder_PG/CW_Folder_PG/Models/alexnet_cnn.pkl"

with open(alex_net, mode ="wb") as f:
    pickle.dump(alexnet_cnn, f)

In [ ]:
alex_net = "/content/drive/MyDrive/CV_Coursework/CW_Folder_PG/CW_Folder_PG/Models/alexnet_cnn.pkl"

with open(alex_net, "rb") as f:
    alex_cnn = pickle.load(f)